In [ ]:
import json
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import re
import os
import sys
import javalang
from xml.dom.minidom import parse
import xml.dom.minidom

In [ ]:
def variable_query(root, name, target_list):
    children = None
    if root is None:
        #print(root)
        return 1
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    for child in children:
        if isinstance(child, (javalang.tree.VariableDeclarator)) and child.name == name:
            target_list.append(child)
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            variable_query(child,name,target_list)

            
def permission_check(permission_list,file_path):
    try:
        DOMTree = xml.dom.minidom.parse(file_path)
    except:
        return False
    collection = DOMTree.documentElement
    permission_tags = collection.getElementsByTagName('uses-permission')
    for tag in permission_tags:
        name = tag.getAttribute("android:name")
        if name in permission_list:
            return False
    return True
def factors_check(name_list,text):
    for name in name_list:
        if name in text:
            return True
    return False

In [ ]:
def walk_tree_nfc(root,violation_list,var_names):
    children = None
    if root is None:
        #print(root)
        return
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    for child in children:
        
        if isinstance(child, (javalang.tree.FieldDeclaration,javalang.tree.LocalVariableDeclaration)):
            if 'Ndef' in child.type.name:
                for declarator in child.declarators:
                    if isinstance(declarator, (javalang.tree.VariableDeclarator)):
                        var_names.append(declarator.name)
        if isinstance(child, javalang.tree.MethodInvocation):
            
            #print(child)
            #print('method ',child.member)
            #print('qualifier ',child.qualifier)
            if child.member == 'createApplicationRecord':
                violation_list.append(child.member)
                
        if isinstance(child, javalang.tree.This):
            for member in child.selectors:
                if isinstance(member, (javalang.tree.MethodInvocation)):
                    #print(member.member)
                    if isinstance(child.selectors[0], javalang.tree.MemberReference):
                        if child.selectors[0].member in var_names and member.member == 'writeNdefMessage':
                            violation_list.append(member.member)
        
        if isinstance(child, (javalang.tree.MethodInvocation)):
            #print('method ',child.member)
            #print('qualifier ',child.qualifier)
            #if child.member == 'takePicture' or child.member == 'createCaptureSession':
            #    violation_list.append('violation')
            if child.qualifier in var_names and child.member == 'writeNdefMessage':
                violation_list.append(child.member)
                
        
        
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            walk_tree_nfc(child,violation_list,var_names)


In [ ]:
def nfc_security_detector(file_path,text,treenode):
    if text == False and treenode == False :
        return 0
    factor_list = ['Ndef','createApplicationRecord','writeNdefMessage']
    if not factors_check(factor_list, text):
        return 0
    
    violation_list = list()
    name_list = list()
    walk_tree_nfc(treenode,violation_list,name_list)
    if 'writeNdefMessage' in violation_list and not 'createApplicationRecord' in violation_list:
        print('nfc_security violation')
        return 1
    return 0


def nfc_SD_detector(root, file):
    permission_list = ['android.permission.NFC','android.permission.NFC_PREFERRED_PAYMENT_INFO','android.permission.NFC_TRANSACTION_EVENT']
    if file == 'AndroidManifest.xml':

        file_path = os.path.join(root, file)
        #if permission_check(permission_list,file_path):
            #print(code_folder, 'has no nfc function')
            #return 0

        try:
            DOMTree = xml.dom.minidom.parse(file_path)
        except:
            return 0
        collection = DOMTree.documentElement
        movies = collection.getElementsByTagName('uses-permission')
        nfc_action_set = set()
        for intent in collection.getElementsByTagName('intent-filter'):
            for action in intent.getElementsByTagName('action'):
                action_name = action.getAttribute("android:name")
                if 'android.nfc.action' in action_name:
                    nfc_action_set.add(action_name)
        if not 'android.nfc.action.NDEF_DISCOVERED' in nfc_action_set and len(nfc_action_set) >0:
            print('nfc_SD violation')
            return 1
    return 0


In [ ]:
i=0

pridir_code = 'D:/jupyterCode/summer_object/apks/decomplied/'
for code_folder in os.listdir(pridir_code)[:]:
    #with open('text.txt','r') as file:
    #    code_folder = file.read()
    if code_folder.endswith(".zip"):
        continue
        
    row = {'apk_name':code_folder,'mtp_SD':0,'hardware_security':0,'nfc_SD':0,'animation_hedonism':0, \
           'media_SD':0,'media_hedonism':0,'media_Uni':0,'nfc_security':0,\
       'telephony_security':0,'telephony_conformity':0}
    full_code_folder = pridir_code+code_folder
    for root, dirs, files in os.walk(full_code_folder):
        for file in files:

            try:
                #nfc
                if not row['nfc_SD']:
                    row['nfc_SD'] = nfc_SD_detector(root, file)

                if file.endswith(".java"):
                    file_path = os.path.join(root, file)

                    try:
                        with open(file_path, "r") as file:
                            text = file.read()
                        treenode = javalang.parse.parse(text)
                    except:
                        text = False
                        treenode = False
                        continue
                    #nfc
                    if not row['nfc_security']:
                        row['nfc_security'] = nfc_security_detector(file_path,text,treenode)

            except:
                print('error')
    i+=1